In [1]:
# 라이브러리 호출
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets 
from torch.nn import parameter #파라미터 목록을 가지고 있는 패키지
from torch import Tensor
import torch.nn.functional as F 
from torch.utils.data import DataLoader 
import math # 수학과 관련되어 다양한 함수들과 상수들이 정의되어 있는 라이브러리 

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
cuda = True if torch.cuda.is_available() else False

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
torch.manual_seed(125)
if torch.cuda.is_available() :
  torch.cuda.manual_seed_all(125)

In [2]:
# 데이터 전처리
import torchvision.transforms as transforms

mnist_transform = transforms.Compose([ 
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5), (1.0, ))])

In [3]:
# 데이터셋 내려받기
from torchvision.datasets import MNIST

download_root  = "../simple-machine-learning-project/Deeplearning-Pytorch-Textbook-main/chap07/MNIST_DATASET"

train_dataset = MNIST(download_root, transform = mnist_transform, train = True, download= True)
"""MNIST 내려받을 위치, 앞에서 설정한 데이터 전처리 적용, train = True면 훈련용 데이터 셋을 False면 테스트용 데이터셋, download True면, MNIST파일이 없으면 내려받고 이미 존재하면 내려받지 않음
"""
valid_dataset = MNIST(download_root, transform = mnist_transform, train = False, download= True)

test_dataset = MNIST(download_root, transform = mnist_transform, train = False, download= True)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../simple-machine-learning-project/Deeplearning-Pytorch-Textbook-main/chap07/MNIST_DATASET\MNIST\raw\train-images-idx3-ubyte.gz to ../simple-machine-learning-project/Deeplearning-Pytorch-Textbook-main/chap07/MNIST_DATASET\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../simple-machine-learning-project/Deeplearning-Pytorch-Textbook-main/chap07/MNIST_DATASET\MNIST\raw\train-labels-idx1-ubyte.gz to ../simple-machine-learning-project/Deeplearning-Pytorch-Textbook-main/chap07/MNIST_DATASET\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../simple-machine-learning-project/Deeplearning-Pytorch-Textbook-main/chap07/MNIST_DATASET\MNIST\raw\t10k-images-idx3-ubyte.gz to ../simple-machine-learning-project/Deeplearning-Pytorch-Textbook-main/chap07/MNIST_DATASET\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../simple-machine-learning-project/Deeplearning-Pytorch-Textbook-main/chap07/MNIST_DATASET\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../simple-machine-learning-project/Deeplearning-Pytorch-Textbook-main/chap07/MNIST_DATASET\MNIST\raw



In [12]:
# 데이터셋 메모리로 가져오기
batch_size = 64
train_loader = DataLoader(dataset = train_dataset, 
                          batch_size = batch_size, 
                          shuffle = True)

valid_loader = DataLoader(dataset = valid_dataset, 
                          batch_size = batch_size, 
                          shuffle = True)
test_loader = DataLoader(dataset = test_dataset, 
                          batch_size = batch_size, 
                          shuffle = True)

In [25]:
print(len(train_dataset))

60000


In [27]:
batch_size = 100
n_iters = 6000
num_epochs = n_iters/(len(train_dataset)/ batch_size) 
num_epochs = int(num_epochs)

In [28]:
print(num_epochs)

10


In [29]:
# # 셀 클래스 구축
# class LSTMCell(nn.Module) : 
#   def __init__(self, input_size, hidden_size, bias = True ) :
#     super(LSTMCell, self).__init__()
#     self.input_size = input_size
#     self.hidden_size = hidden_size
#     self.bias = bias
#     self.x2h = nn.Linear(input_size, 4*hidden_size, bias = bias) 
#     """ 모든 게이트는 이 코드로 구할 수 있음 """
#     self.h2h = nn.Linear(hidden_size, 4*hidden_size, bias = bias)
#     self.reset_parameters()
    
#     def reset_parameters(self) : 
#       """모델 파라미터 초기화 """
#       std = 1.0 / math.sqrt(self.hidden_size)
#       for w in self.parameters() :
#         w.data.uniform_(-std, std)
        
#     def forward(self, x, hidden) :
#       hx, cx = hidden
#       x = x.view(-1, x.size(1))
      
#       gates = self.x2h(x) + self.h2h(hx)
#       gates = gates.squeeze() 
#       """ 
#       차원 줄일 때 2차원 -> 1차원으로 줄임
#       """
#       ingate, forgetgate, cellgate, outgate = gates.chunk(4,1)
      
#       ingate = F.sigmoid(ingate)
#       forgetgate = F.sigmoid(forgetgate)
#       cellgate = F.tanh(cellgate)
#       outgate = F.sigmoid(outgate)
      
#       cy = torch.mul(cx, forgetgate) + torch.mul(ingate, cellgate)
#       hy = torch.mul(outgate, F.tanh(cy))
#       """ mul은 (적용 텐서, 얼마 곱할건지)로 구성되며, 텐서의 곱을 할 때 사용함
#       """
#       return (hy, cy) 

In [49]:
# 셀 클래스 구축
class LSTMCell(nn.Module): 
    def __init__(self, input_size, hidden_size, bias=True):
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias
        self.x2h = nn.Linear(input_size, 4 * hidden_size, bias=bias) 
        """ 모든 게이트는 이 코드로 구할 수 있음 """
        self.h2h = nn.Linear(hidden_size, 4 * hidden_size, bias=bias) 
        self.reset_parameters()

    def reset_parameters(self):  
        """모델 파라미터 초기화 """
        std = 1.0 / math.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std, std) 

    def forward(self, x, hidden):
        hx, cx = hidden
        x = x.view(-1, x.size(1))

        gates = self.x2h(x) + self.h2h(hx) 
        gates = gates.squeeze() 
        """ 
#       차원 줄일 때 2차원 -> 1차원으로 줄임
#       """
        ingate, forgetgate, cellgate, outgate = gates.chunk(4, 1) 

        ingate = F.sigmoid(ingate) 
        forgetgate = F.sigmoid(forgetgate) 
        cellgate = F.tanh(cellgate) 
        outgate = F.sigmoid(outgate) 

        cy = torch.mul(cx, forgetgate) + torch.mul(ingate, cellgate) 
        hy = torch.mul(outgate, F.tanh(cy)) 
        return(hy, cy)
    """ mul은 (적용 텐서, 얼마 곱할건지)로 구성되며, 텐서의 곱을 할 때 사용함
    """

중요한 것은 
- self.x2h = nn.Linear(input_size, 4*hidden_size, bias = bias)
-  gates = self.x2h(x) + self.h2h(hx)

  - x2h input_size : 입력층으로 입력되는 훈련 데이터셋의 특성수(칼럼개수)
  - x2h 4*hidden_size : 입력층과 은닉층 사이의 가중치
  - bias : 입력층과 은닉층 사이의 편향
  - h2h input_size : 은닉층의 뉴런, 유닛개수(은닉층의 특성수)
  - h2h 4*hidden_size : 은닉층과 은닉층 사이의 가중치
  - bias : 은닉층과 은닉층 사이의 바이어스

#### 이렇게 계산된 게이트는 4개의 게이트로 chunk()라는 함수를 이용해 분할 되서 각각 저장됨. 일반적으로는 bias에도 4를 곱해주지만 여기선 은닉층의 뉴런, 유닛개수만 4를 곱해줌

- chunk : 텐서 쪼갤 때 사용하는 함수. 
- 4 : 몇개로 쪼갤지
- 1 : 어떤 차원을 기준으로 쪼갤지. dim = 1이면 열단위로 텐서를 분할하겠다
- uniform() : 난수 생성. -> randint(), random()등등을 사용할 수 있음 

ri = randint(1, 10) ------ 1부터 10 사이의 임의의 정수
print(ri)

rd = random() ------ 0부터 1 사이의 임의의 실수(float)
print(rd)

ui = uniform(1, 10) ------ 1부터 10 사이의 임의의 실수(float)
print(ui)

rr = randrange(1, 10, 2) ------ 1부터 10 사이를 2씩 건너뛴 임의의 정수(예 1, 3, 5, 7, 9)
print(rr)

In [54]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim, bias=True):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim #은닉층의 뉴런/유닛 개수

        self.layer_dim = layer_dim
        self.lstm = LSTMCell(input_dim, hidden_dim, layer_dim) 
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        if torch.cuda.is_available(): # GPU 사용 유무 확인
            h0 = torch.tensor(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda()) #(은닉층의 계층 개수, 배치 크기, 은닉층의 뉴런 개수) 형태를 갖는 은닉 상태를 0으로 초기화
        else:
            h0 = torch.tensor(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))

        if torch.cuda.is_available(): # GPU 사용 유무 확인
            c0 = torch.tensor(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda()) # (은닉층의 계층 개수, 배치 크기, 은닉층의 뉴런 개수) 형태를 갖는 셀 상태를 0으로 초기화
        else:
            c0 = torch.tensor(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))

        outs = []
        cn = c0[0,:,:] # (은닉층의 계층 개수, 배치 크기, 은닉층의 뉴런 개수) 크기를 갖는 셀 상태에 대한 텐서
        hn = h0[0,:,:] # (은닉층의 계층 개수, 배치 크기, 은닉층의 뉴런 개수) 크기를 갖는 은닉 상태에 대한 텐서

        for seq in range(x.size(1)): # LSTM 셀 계층을 반복하여 쌓아 올립니다.
            hn, cn = self.lstm(x[:,seq,:], (hn,cn)) # 은닉 상태(hn)와 셀 상태를 LSTMCell에 적용한 결과를 또다시 hn, cn에 저장
            outs.append(hn)

        out = outs[-1].squeeze()
        out = self.fc(out)
        return out

In [55]:
# 옵티마이저, 손실함수 지정
input_dim = 28
hidden_dim = 128
layer_dim = 1
output_dim = 10

model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
if torch.cuda.is_available(): 
  model.cuda()
  
criterion = nn.CrossEntropyLoss() # 중요
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(),  lr=learning_rate) # 중요

In [56]:
seq_dim = 28
loss_list = []
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader): # 훈련 데이터셋을 이용한 모델 학습
        if torch.cuda.is_available(): #GPU 사용 유무 확인
            images = torch.tensor(images.view(-1, seq_dim, input_dim).cuda()) 
            labels = torch.tensor(labels.cuda())
        else: # GPU를 사용하지 않기 때문에 else 구문이 실행
            images = torch.tensor(images.view(-1, seq_dim, input_dim))
            labels = torch.tensor(labels)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels) # 손실 함수를 이용하여 오차 계산

        if torch.cuda.is_available():
            loss.cuda()

        loss.backward()
        optimizer.step() #파라미터 업데이트
        loss_list.append(loss.item())
        iter += 1

        if iter % 500 == 0: # 정확도(accuracy) 계산
            correct = 0
            total = 0
            for images, labels in valid_loader: # 검증 데이터셋을 이용한 모델 성능 검증

                if torch.cuda.is_available():
                    images = torch.tensor(images.view(-1, seq_dim, input_dim).cuda())
                else:
                    images = torch.tensor(images.view(-1, seq_dim, input_dim))

                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1) #모델을 통과한 결과의 최댓값으로부터 예측 결과 가져오기

                total += labels.size(0) # 총 레이블 수
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()

            accuracy = 100 * correct / total
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

C:\Users\hop09\AppData\Local\Temp\ipykernel_2792\1942578715.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = torch.tensor(images.view(-1, seq_dim, input_dim))
C:\Users\hop09\AppData\Local\Temp\ipykernel_2792\1942578715.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)
C:\Users\hop09\AppData\Local\Temp\ipykernel_2792\3255531632.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  h0 = torch.tensor(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))
C:\Users\hop09\AppData\Local\Temp\ipykern

Iteration: 500. Loss: 2.2557575702667236. Accuracy: 22.639999389648438
Iteration: 1000. Loss: 0.7829233407974243. Accuracy: 70.83999633789062
Iteration: 1500. Loss: 0.4939519464969635. Accuracy: 87.95999908447266
Iteration: 2000. Loss: 0.2981965243816376. Accuracy: 93.0199966430664
Iteration: 2500. Loss: 0.23295961320400238. Accuracy: 94.08000183105469
Iteration: 3000. Loss: 0.3366606533527374. Accuracy: 96.02999877929688
Iteration: 3500. Loss: 0.1180586963891983. Accuracy: 96.55999755859375
Iteration: 4000. Loss: 0.05988272652029991. Accuracy: 96.37999725341797
Iteration: 4500. Loss: 0.08962131291627884. Accuracy: 96.7699966430664
Iteration: 5000. Loss: 0.06190209090709686. Accuracy: 96.93000030517578
Iteration: 5500. Loss: 0.009094631299376488. Accuracy: 97.62999725341797
Iteration: 6000. Loss: 0.04170197993516922. Accuracy: 97.37999725341797
Iteration: 6500. Loss: 0.034373193979263306. Accuracy: 97.66000366210938
Iteration: 7000. Loss: 0.2412298321723938. Accuracy: 97.04000091552734

In [57]:
# 모델 예측 성능 확인
def evaluate(mode, val_iter) :
  corrects, total, total_loss = 0,0,0
  model.eval()
  for images, labels in val_iter :
    if torch.cuda.is_available() :
      images = torch.tensor(images.view(-1, seq_dim, input_dim).cuda())
      
    else :
      images = torch.tensor(images.view(-1, seq_dim, input_dim)).to(device)
      
    logit = model(images).to(device)
    loss = F.cross_entropy(logit, labels, reduction = "sum")
    _, predicted = torch.max(logit.data, 1)
    
    total += labels.size(0)
    total_loss += loss.item()
    corrects += (predicted == labels).sum()
    
  avg_loss = total_loss/ len(val_iter.dataset)
  avg_accuracy = correct / total
  return avg_loss, avg_accuracy